In [1]:
%reload_ext autoreload
%autoreload 2

import dp_relational
import dp_relational.data.movies
import dp_relational.lib.synth_data

c:\C\MIT\Spring24\UROP_P2\code\code\dp_relational\data\movies.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies_df = pd.read_csv(file_path, delimiter='::', header=None, names=['ID', 'Title', 'Genres'], encoding='latin1', index_col=[0])
c:\C\MIT\Spring24\UROP_P2\code\code\dp_relational\data\movies.py:37: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users_df = pd.read_csv(file_path, delimiter='::', header=None, names=['ID', 'Gender', "Age", "Occupation", "Zipcode"], encoding='latin1', index_col=[0])
c:\C\MIT\Spring24\UROP_P2\code\code\dp_relational\data\movies.py:42: ParserWarning: Falling back 

In this notebook, I demonstrate the sheer magnitude of the speedups I have implemented.

\- Ojas Gulati

In [2]:
dataset = dp_relational.data.movies.dataset

# parameters

n_syn1=776
n_syn2=1208

epsilon=3.0
eps1=1.0
eps2=1.0
eps_rel = epsilon - eps1 - eps2

k=2
dmax=10 # TODO: kinda urgently, this one is a mess
dataset = dataset(dmax)

synth='mst'
verbose = True

In [3]:
rel_dataset = dataset
df1_synth = dp_relational.lib.synth_data.compute_single_table_synth_data(rel_dataset.table1.df, n_syn1, synth, epsilon=eps1)
df2_synth = dp_relational.lib.synth_data.compute_single_table_synth_data(rel_dataset.table2.df, n_syn2, synth, epsilon=eps2)

df_rel = rel_dataset.df_rel

c:\C\MIT\Spring24\UROP_P2\code\code\.venv\lib\site-packages\mbi\__init__.py:15: UserWarning: MixtureInference disabled, please install jax and jaxlib
  warnings.warn('MixtureInference disabled, please install jax and jaxlib')


Fitting with 262144 dimensions
Getting cliques
Estimating marginals
Fitting with 294 dimensions
Getting cliques
Estimating marginals


External synthetic data generation takes 1:40 on my machine

In [18]:
qm = dp_relational.lib.synth_data.QueryManagerBasic(rel_dataset, k=k, df1_synth=df1_synth, df2_synth=df2_synth)

100%|██████████| 1208/1208 [00:36<00:00, 33.09it/s]


Initializing the Q matrix takes a very long time (about 2:30 min) and will often fail due to lack of memory: it is at least 7.6GB here (I know this from error messages)

In [25]:
T = 3
b_round = dp_relational.lib.synth_data.learn_relationship_vector_basic(qm, 1, T=T, verbose=verbose)
relationship_syn = dp_relational.lib.synth_data.make_synthetic_rel_table(qm, b_round)
ave_error, max_error = dp_relational.lib.synth_data.evaluate_synthetic_rel_table(qm, relationship_syn)
ave_error, max_error

 67%|██████▋   | 2/3 [01:05<00:32, 32.92s/it]


MemoryError: Unable to allocate 1.23 GiB for an array with shape (937408, 176) and data type float64

This took 16:00 to run for 10 iterations.

In [26]:
qm_torch = dp_relational.lib.synth_data.QueryManagerTorch(rel_dataset, k=k,
                                df1_synth=df1_synth, df2_synth=df2_synth, verbose=verbose)
qm_torch.num_relationship *= 5

In [30]:
T = 3
b_round = dp_relational.lib.synth_data.learn_relationship_vector_torch(
    qm_torch, eps_rel * 4, T=T, verbose=verbose, T_mirror=50)
df_syn_rel = dp_relational.lib.synth_data.make_synthetic_rel_table_sparse(qm_torch, b_round)
ave_error, max_error = dp_relational.lib.synth_data.evaluate_synthetic_rel_table(qm_torch, df_syn_rel)
ave_error, max_error

 33%|███▎      | 1/3 [00:03<00:07,  3.67s/it]


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 59994112 bytes.

In [28]:
qm_torch.num_relationship

2010

Meanwhile, the full QM-Torch training with 10 iters is done in 3 minutes.

In [16]:
import numpy as np
100 * np.sum(np.abs(qm.rand_ans - qm.true_ans)) / len(qm.true_ans)

num_relationship = qm.rel_dataset.df_rel.shape[0]
true_ans = np.zeros(qm.num_all_queries)


# TODO: need to improve it!
for i in range(num_relationship):
    
    # TODO: assert ID_1 and ID_2 correspond to index
    
    ID1 = qm.rel_dataset.df_rel.iloc[i][qm.rel_dataset.rel_id1_col]
    
    ID2 = qm.rel_dataset.df_rel.iloc[i][qm.rel_dataset.rel_id2_col]
    
    for w in qm.workload_names:
        cols1 = w[0]
        cols2 = w[1]
        v1 = []
        for c1 in cols1:
            v1.append(qm.rel_dataset.table1.df.iloc[ID1][c1])
            
        v2 = []
        for c2 in cols2:
            v2.append(qm.rel_dataset.table2.df.iloc[ID2][c2])
        
        ind = qm.query_ind(w, [v1,v2])
        true_ans[int(ind)] += 1
        
true_ans = true_ans/num_relationship

array([2.68784119e-01, 6.40000000e-01, 1.60794045e-02, ...,
       4.96277916e-04, 1.98511166e-04, 8.93300248e-04])

In [17]:
true_ans - qm.true_ans

array([0., 0., 0., ..., 0., 0., 0.])

In [12]:
T = 30
b_round = dp_relational.lib.synth_data.learn_relationship_vector_torch_masked(
    qm_torch, eps_rel, T=T, verbose=verbose)
df_syn_rel = dp_relational.lib.synth_data.make_synthetic_rel_table_sparse(qm_torch, b_round)
ave_error, max_error = dp_relational.lib.synth_data.evaluate_synthetic_rel_table(qm_torch, df_syn_rel)

ave_error, max_error

100%|██████████| 30/30 [00:47<00:00,  1.59s/it]


(1.6702276971616152, 16.422165474154205)